## Diving Deeper into Pandas

* So far we have only really scratched the surface of what Pandas can do
* Merging Datasets together
* Multi-level indexing
* Split-Apply-Combine
* Working with Timeseries

### Merging Data

* Bringing disparate datasets together is one of the more powerful features of Pandas
* Like with Python lists, you can `append()` and `concat()` Pandas `Series` and `Dataframes`
* These functions work best for simple cases

In [ ]:
# concatinate two series together
ser1 = pd.Series(['A', 'B', 'C'], index=[1, 2, 3])
ser2 = pd.Series(['D', 'E', 'F'], index=[4, 5, 6])
pd.concat([ser1, ser2])

In [ ]:
# concatinate two dataframes
df1 = pd.DataFrame({"A":["A1", "A2"],
                    "B":["B1","B2"]},index=[1,2])
df2 = pd.DataFrame({"A":["A3", "A4"],
                    "B":["B3","B4"]},index=[3,4])
pd.concat([df1,df2])

In [ ]:
# concatinate dataframes horizontally
df1 = pd.DataFrame({"A":["A1", "A2"],
                    "B":["B1","B2"]},index=[1,2])
df2 = pd.DataFrame({"C":["C1", "C2"],
                    "D":["D1","D2"]},index=[1,2])
pd.concat([df1,df2], axis=1)

In [ ]:
# What happens when indexes don't line up
df1 = pd.DataFrame({"A":["A1", "A2"],
                    "B":["B1","B2"]},index=[1,2])
df2 = pd.DataFrame({"A":["A3", "A4"],
                    "B":["B3","B4"]},index=[3,4])
pd.concat([df1,df2], axis=1)

In [ ]:
# create a hierarchical index
df1 = pd.DataFrame({"A":["A1", "A2"],
                    "B":["B1","B2"]},index=[1,2])
df2 = pd.DataFrame({"A":["A3", "A4"],
                    "B":["B3","B4"]},index=[3,4])
pd.concat([df1,df2], keys=["df1", 'df2'])

### Merging and Joining

* While `concat()` is useful it lacks the power to do complex data merging
* For example, I have two tables of different data but one overlapping column
* This is where the `merge()` function becomes useful because it lets you *join* datasets
* The concept of "join" has lots of theory and is a richly developed method for *joining* data

#### One-to-one joins

In [ ]:
# create two dataframes with one shared column
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})

In [ ]:
# display df1
df1

In [ ]:
# display df2
df2

In [ ]:
# merge df1 and df2 into a new dataframe df3
df3 = pd.merge(df1, df2)
df3

* The new dataframe `df3` now has all of the data from df1 and df2
* The `merge` function automatically connected the two tables on the "employees" column
* But what happens when your data don't line up?

#### Many-to-one joins

* Sometimes there isn't a one to one relationshp between rows in the two datasets
* A *many-to-one* join lets you combine these datasets

In [ ]:
df3

In [ ]:
# make another dataframe about the supervisor for each group
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                    'supervisor': ['Carly', 'Guido', 'Steve']})
df4

In [ ]:
# Merge df3 from above with the supervisor info in df4
pd.merge(df3,df4)

* Notice how the information about Guido, the manager for Engineering, is repeated.
* While this might seem like duplicated data, it makes it easier to quickly look up Jake and Lisa's supervisor without consulting multiple tables

#### Many-to-many joins

* Let's combine the employee information with skills information
* Notice there isn't a one to one or even a one to many relationship between these tables
* Each group can have multiple skills, so **what do you think will happen?**

In [ ]:
# Use the employee table specified above
df1

In [ ]:
# create a new dataframe with skills information
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                              'Engineering', 'Engineering', 'HR', 'HR', 'Librarian'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux',
                               'spreadsheets', 'organization', 'nunchucks']})
df5

In [ ]:
pd.merge(df1, df5)

* Amazing, Pandas merge capabilities are very useful
* But what do you do if the names of your columns don't match?
* You could change column names...
* But that is crazy! Just use the `left_on` and `right_on` parameters to the `merge()` function

In [ ]:
# Use the employee table specified above
df1

In [ ]:
# Create a new salary table, but use "name" instead of "employee" for the column index
df3 = df3 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'salary': [70000, 80000, 120000, 90000]})
df3

In [ ]:
# lets try and merge them without specifying what to merge on
pd.merge(df1, df3)

* What are the column names I should specify?

In [ ]:
# Now lets specify the column name 
pd.merge(df1, df3, left_on="employee", right_on="name" )

* Notice we now have a redundant employee/name column, this is a by-product of merging different columns
* If you want to get rid of it you can use the `drop` method

In [ ]:
# drop the name column, axis=1 means axis='col', which is confusing
pd.merge(df1, df3, left_on="employee", right_on="name" ).drop("name", axis=1)

### Splitting Data with GroupBy


* A common pattern in data analysis is splitting data by a key and then performing some math on all of the values with that key and finally combining it all back together
* This is commonly known in data circles as *split, apply, combine*


In [ ]:
# create a dataframe to illustrate GroupBy
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data': range(6),
                   'things':[45,234,6,2,1324,345]}, columns=['key', 'data', 'things'])
df

In [ ]:
# Dataframes have a method, groupby(), that takes a column name be be the grouping key
df.groupby('key')

* Cool, but what is that? Well, we need to tell Pandas what to *do* with the groups
* This is where we get to the *apply* step
* We need to specify what kind of aggregation, transformation, or computation to perform on the group

In [ ]:
# Tell pandas to add up all of the values for each key
df.groupby('key').sum()

In [ ]:
grouped_dataframe = df.groupby('key')
grouped_dataframe.sum()

The following table summarizes some other built-in Pandas aggregations:

| Aggregation              | Description                     |
|--------------------------|---------------------------------|
| ``count()``              | Total number of items           |
| ``size()``               | Total number of items w/ NaNs   |
| ``first()``, ``last()``  | First and last item             |
| ``mean()``, ``median()`` | Mean and median                 |
| ``min()``, ``max()``     | Minimum and maximum             |
| ``std()``, ``var()``     | Standard deviation and variance |
| ``mad()``                | Mean absolute deviation         |
| ``prod()``               | Product of all items            |
| ``sum()``                | Sum of all items                |

These are all methods of ``DataFrame`` and ``Series`` objects.

#### Exercise
* Create a new dataframe with different keys and values, then try the aggregations above

In [ ]:
# Modify this dataframe to put your own values in
df = pd.DataFrame({'key': [???],
                   'data': [???]}, columns=['key', 'data'])

df.groupby("key").???

### Working with Time

* One of the most powerful features of Pandas is its time series functionality
* Dates and time are a Python and Pandas data type (like integers and strings)
* By using the `datetime` data types you can do advanced, time-centric analysis
* One thing to remember about computers is they are *very* specific
    * *Time stamps* - a specific moment in time (July 4th, 2017 at 7:52am and 34 seconds)
    * *Time intervals* - a length of time with start and end points (The year 2017)
    * *Time duration* - a specific length of time (a year, a month, a day)

In [ ]:
# Datetime in pure Python
import datetime

date = datetime.datetime(year=2017, month=6, day=13)
date

In [ ]:
type(date)

In [ ]:
# what is that date's month?
date.month

In [ ]:
# what is that date's day?
date.day

In [ ]:
# use the parser function in the datautil library to parse human dates
from dateutil import parser
date = parser.parse("4th of July, 2017")
date

In [ ]:
# get the month
date.month

#### Exercise

Try some different date strings, see how smart Python can be.

In [ ]:
my_date = parser.parse("<your date string here")
my_date

* You can use [*string format codes*](https://docs.python.org/3/library/datetime.html#strftime-strptime-behavior) for printing dates and time in different formats (especially useful for making human readable dates)
* Pass a format string to the `strftime()` method to print out a pretty date

In [ ]:
# Get the weekday 
date.strftime("%A")

In [ ]:
date.strftime("%B")

In [ ]:
## Try some of the different string format codes and see what happens
date.

In [ ]:
## Try combining a few of them together with punctuation too
date.

### Working with time in Pandas

* Just like how Pandas has its own datatypes for numbers, Pandas has its own dates and times (to support more granularity)
* If you have a lot of dates, it is often useful to use the Pandas functions over the native Python functions
* Pandas is most powerful when you index by time using the `DatetimeIndex`

In [ ]:
# Create a Series with a DateTime index
index = pd.DatetimeIndex(['2014-03-04', '2014-08-04',
                          '2015-04-04', '2015-09-04',
                          '2016-01-01', '2016-02-16'])
data = pd.Series([0, 1, 2, 3, 4, 5], index=index)
data

* Now that the index is made of DateTimes we can index using date strings
* Note, this only works on strings

In [ ]:
# grab the value for a specific day
data["2015-04-04"]

In [ ]:
# grab a slice between two dates
data['2014-08-01':'2016-01']

In [ ]:
# give me everything from 2015
data['2015']

* Pandas has some functions to make parsing dates easy too

In [ ]:
# use the to_datetime function instead of the parser function
date = pd.to_datetime("4th of July, 2017")
date

In [ ]:
# use string format codes to get the weekday
date.strftime("%A")

In [ ]:
# give me today's date
today = pd.to_datetime("today")
today

* That is the day, but also the exact time... 
* Timestamps must always be a specific moment

#### Exercise
* Use the [*string format codes*](https://docs.python.org/3/library/datetime.html#strftime-strptime-behavior) to print today's date in the "YYYY-MM-DD" format. HINT: You will have to combine multiple codes and dashes

In [ ]:
# Replace the ??? with the write string format code
print(today.strftime("???"))

### Playing with time on real data

* Let's look at the [311 data for the city of Pittsburgh](https://data.wprdc.org/dataset/311-data) from the WPRDC
* Did you know, you can give the URL directly to Pandas!

In [ ]:
# load the 311 data directly from the WPRDC
pgh_311_data = pd.read_csv("https://data.wprdc.org/datastore/dump/76fda9d0-69be-4dd5-8108-0de7907fc5a4")
pgh_311_data.head()

In [ ]:
# Inspect the dataframe and Pandas automatic data type detection
pgh_311_data.info()

* Ok, now we have the data, but we need it to be indexed by date
* **What column has the date information?**
* **What format do you think that column is currently in?**
* **What function might we use to convert that column into dates?**

In [ ]:
pgh_311_data['CREATED_ON'].head()

In [ ]:
# convert the "CREATED_ON" column to dates
pd.to_datetime(pgh_311_data['CREATED_ON']).head()

* We can convert the "CREATED_ON" column to Pandas `datetime` objects
* Now we have to set that to the dataframe's index

In [ ]:
# set the index of pgh_311_data to be the parsed dates in the "CREATED_ON" column
pgh_311_data.index = pd.to_datetime(pgh_311_data['CREATED_ON'])
pgh_311_data.head()

* Do'h, now we have CREATED_ON twice, that isn't very tidy
* We can also skip this extra conversion step entirely by specifying the index column and date parsing in `read_csv()` function call.

In [ ]:
# load the 311 data directly from the WPRDC and parse dates directly
pgh_311_data = pd.read_csv("https://data.wprdc.org/datastore/dump/76fda9d0-69be-4dd5-8108-0de7907fc5a4",
                           index_col="CREATED_ON", 
                           parse_dates=True)
pgh_311_data.head()

In [ ]:
pgh_311_data.info()

* Now that the dataframe has been indexed by time we can select 311 complains by time

In [ ]:
# Select 311 complaints on January 1st, 2016
pgh_311_data['2016-01-01']

In [ ]:
# Select the times just around the new years celebration
pgh_311_data["2015-12-31 20:00:00":"2016-01-01 02:00:00"]

* Someone clearly had a very roudy new years 

#### Exercise

* Using the timeseries index selection, select the complaints made today 
* Next, try and write your code so it will work on any day you execute it
    * *hint*: try the `pd.datetime('today')` 
    * *Another hint*: Remember the DateTime gives you the exact time 
    * *Yet another hint*: Datetime indexing only works with string representations 

In [ ]:
# Write your code here
pgh_311_data[]

In [ ]:


# create a Pandas datetime for today
today = pd.to_datetime("today")
formatted_today_string = today.strftime("%Y-%m-%d")
print(today)
print(formatted_today_string)

# use Pandas date string indexing to retrieve all rows for this today's date
todays_311s = pgh_311_data[formatted_today_string]
todays_311s

### Grouping time with the resample method

* Instead of using the `groupby()` method, you use the `resample()` method to *split* time into groups
* Then you can *apply* the regular aggregation functions 

In [ ]:
# compute the mean of complaints per quarter...note this doesn't make sense, but works anyway
pgh_311_data.resample("Q").mean()

In [ ]:
# count the number of complaints per month
pgh_311_data.resample("M").count()

* Ok, these data are *begging* to be visualized, so I'm going to give you a teaser of next week 

In [ ]:
# load up the data visualization libraries
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn; seaborn.set()

In [ ]:
# Create a graph of the monthly complaint counts
pgh_311_data['REQUEST_ID'].resample("M").count().plot()

Try the code above, but re-sampling based upon different date periods. The strings for specifying an offset are located [here](http://pandas.pydata.org/pandas-docs/stable/timeseries.html#offset-aliases)


In [ ]:
# Try a different resampling here



In [ ]:
# Try yet another resampling here



* OK, we've done some "fun" stuff with Time, but maybe we want to start doing deeper analysis
* To do that, we need to know what all these columns mean?
* Fortunately, this dataset has a [data dictionary](https://data.wprdc.org/dataset/311-data/resource/d3e98904-4a86-45fb-9041-0826ab8d56d0), which provides a bit more information.

### Exploring the 311 Data

* Now we can use what we have learned to do some exploratory data analysis on the 311 data
* First, lets use the `sample()` method to grab 10 random rows so we can get a feel for the data


In [ ]:
# Sample 10 random rows from the dataframe
pgh_311_data.sample(10)

#### Exercise


* What are the possible *origins* of complaints?
* How many complaints are coming from each source?

*HINT*: Scroll back up to the top of to look at the Dataframes refresher.

In [ ]:
pgh_311_data['REQUEST_ORIGIN'].value_counts()

#### Exercise

* *Group* the complaints *by* neighborhood and get the *size* of each group

In [ ]:
pgh_311_data.groupby('NEIGHBORHOOD').size()

In [ ]:
# Note, for just counting the groupby and value_counts are equivalent
# There is more than one way to skin the cat (or panda)
pgh_311_data['NEIGHBORHOOD'].value_counts()

#### Exercise

* *Group* the complaints *by* type and get the *count* for each group

In [ ]:
pgh_311_data.groupby("REQUEST_TYPE")['REQUEST_TYPE'].count()

This categorical data is far too granular. 
Fortunately, if we look at the [311 Data](https://data.wprdc.org/dataset/311-data) we can see there is a [311 Issue and Category Codebook](https://data.wprdc.org/dataset/311-data/resource/40ddfbed-f225-4320-b4d2-7f1e09da72a4). Click on that link and check out the Google Sheets preview of that data.

https://data.wprdc.org/dataset/311-data/resource/40ddfbed-f225-4320-b4d2-7f1e09da72a4

What we need to do is download the CSV from Google Sheets directly into a Pandas dataframe, but this is actually a bit tricky because Google won't easily give us a link to the CSV file.

In [ ]:
# I googled "pandas dataframe from google sheets"
# and found a solution on Stackoverflow
# https://stackoverflow.com/a/35246041
issue_category_mapping = pd.read_csv('https://docs.google.com/spreadsheets/d/' + 
                   '1DTDBhwXj1xQG1GCBKPqivlzHQaLh2HLd0SjN1XBPUw0' +
                   '/export?gid=0&format=csv')
issue_category_mapping.head(5)  # Same result as @TomAugspurger

#### Exercise

* Merge the `pgh_311_data` with the `issue_category_mapping` so we can count the number of complaints per category
* *HINT*: You will need to specify the `left_on` and `right_on` parameters

In [ ]:
# create a new merged dataframe
merged_311_data = pd.merge(pgh_311_data, 
         issue_category_mapping,
         left_on="REQUEST_TYPE",
         right_on="Issue")

merged_311_data.head()

In [ ]:
# get rid of redundant columns
merged_311_data.drop(['Definition','Department', 'Issue'], 
                     axis=1, 
                     inplace=True)
merged_311_data.head()

#### Exercise

* Now that we have category data, count the number of complaints by category

In [ ]:
merged_311_data.groupby("Category")['Category'].count().sort_values(ascending=False)

In [ ]:
merged_311_data.groupby("Category").size().sort_values(ascending=False)

* Selecting data in a Dateframe


In [ ]:
# Select only rows where NEIGHBORHOOD equals "Greenfield" and then count how many complaints came from each source
merged_311_data[merged_311_data['NEIGHBORHOOD'] == 'Greenfield'].groupby('REQUEST_ORIGIN').size()

## Split, Apply, Combine with numeric data

* The 311 complaints are mainly categorical data, which doesn't let use do more mathematical aggregations
* Lets grab a different dataset from the WPRDC, the [Allegheny County Jail Daily Census](https://data.wprdc.org/dataset/allegheny-county-jail-daily-census)

In [ ]:
# Grab three months of data
january17_jail_census = pd.read_csv("https://data.wprdc.org/datastore/dump/3b5d9c45-b5f4-4e05-9cf1-127642ad1d17",
                                  parse_dates=True,
                                  index_col='Date')
feburary17_jail_census = pd.read_csv("https://data.wprdc.org/datastore/dump/cb8dc876-6285-43a8-9db3-90b84eedb46f",
                                   parse_dates=True,
                                  index_col='Date')
march17_jail_census = pd.read_csv("https://data.wprdc.org/datastore/dump/68645668-3f89-4831-b1de-de1e77e52dd3",
                                parse_dates=True,
                                index_col='Date')


In [ ]:
january17_jail_census.head()

In [ ]:
# Use the concat function to combine all three into one dataframe
# Remember I need to make a list of the all the dataframes for
# the concat fuction
jail_census = pd.concat([january17_jail_census, 
                         feburary17_jail_census, 
                         march17_jail_census])
jail_census

In [ ]:
# remove the "_id" column because it is not useful
jail_census.drop("_id", axis=1, inplace=True)
jail_census

In [ ]:
# get just the first day in Feburary 2017
jail_census.loc["2017-02-01"]

In [ ]:
# Compute the average age ate booking by gender for Febuary 1st, 2017
jail_census.loc['2017-02-01'].groupby('Gender')['Age at Booking'].mean()

In [ ]:
# compute the average age at booking by race for Febuary 1st, 2017
jail_census.loc['2017-02-01'].groupby('Race')['Age at Booking'].mean()

If we look at the [data dictionary](https://data.wprdc.org/dataset/allegheny-county-jail-daily-census/resource/f0550174-16b0-4f6e-88dc-fa917e74b56c) we can see the following mapping for race categories
```
Race of Inmate
A-ASIAN OR PACIFIC ISLANDER
B-BLACK OR AFRICAN AMERICAN
H-HISPANIC 
I-AMERICAN INDIAN OR ALASKAN NATIVE
U-UNKNOWN
W-WHITE
```
The `x` category hasn't been described.

In [ ]:
# how many total rows in the dataset have "x" for race
jail_census['Race'].value_counts()['x']

In [ ]:
# Get the statistical summary of age at booking by gender for Febuary 1st, 2017
jail_census.loc['2017-02-01'].groupby('Gender')['Age at Booking'].describe()

In [ ]:
# Compute the difference between Age at Booking and current age
age_difference = jail_census.loc['2017-02-01']['Current Age'] - jail_census.loc['2017-02-01']['Age at Booking']
age_difference.value_counts()

In [ ]:
# Compute the average age for each day
jail_census.resample("D").mean()

In [ ]:
# What is with that NaNs?
jail_census.loc['2017-03-19']

In [ ]:
# visualize the number of inmates
jail_census.resample("D").size().plot()